In [0]:
%tensorflow_version 2.x

import os
import tensorflow as tf
import numpy as np
from prepare_data2 import parse_seq
import pickle

In [3]:
!python3 prepare_data2.py shakespeare_input.txt shake \\n\\n+ -m 500

2020-06-04 09:13:03.323831: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Split input into 31022 sequences...
Longest sequence is 3094 characters. If this seems unreasonable, consider using the maxlen argument!
Removing sequences longer than 500 characters...
29429 sequences remaining.
Longest remaining sequence has length 499.
Removing length-0 sequences...
29429 sequences remaining.
Serialized 100 sequences...
Serialized 200 sequences...
Serialized 300 sequences...
Serialized 400 sequences...
Serialized 500 sequences...
Serialized 600 sequences...
Serialized 700 sequences...
Serialized 800 sequences...
Serialized 900 sequences...
Serialized 1000 sequences...
Serialized 1100 sequences...
Serialized 1200 sequences...
Serialized 1300 sequences...
Serialized 1400 sequences...
Serialized 1500 sequences...
Serialized 1600 sequences...
Serialized 1700 sequences...
Serialized 1800 sequences...
Serialized 1900 sequences..

In [4]:
bs = 128
seq_len = 500
data = tf.data.TFRecordDataset("shake.tfrecords")
data = data.map(lambda x: parse_seq(x))
data = data.shuffle(46000).padded_batch(128, seq_len, drop_remainder=True).repeat()

vocab = pickle.load(open("shake_vocab", mode="rb"))
vocab_size = len(vocab)
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}
ch_to_ind = {ch: ind for (ch, ind) in vocab.items()}

print(vocab_size)
print(vocab)

70
{'w': 3, 'p': 4, ',': 5, 'S': 6, 'i': 7, 'O': 8, 'k': 9, ';': 10, 'n': 11, 'L': 12, 'C': 13, 'j': 14, 'h': 15, 'r': 16, 'W': 17, 'H': 18, 'c': 19, 't': 20, 'g': 21, 'y': 22, 'T': 23, '$': 24, 'A': 25, 'D': 26, 'v': 27, 'M': 28, 'x': 29, '!': 30, '-': 31, 'z': 32, ' ': 33, 'J': 34, 'E': 35, 'd': 36, 'o': 37, 'U': 38, '.': 39, '[': 40, 'B': 41, 'F': 42, 'K': 43, '\n': 44, 'l': 45, 'f': 46, 'Y': 47, 's': 48, 'V': 49, '3': 50, 'q': 51, 'Z': 52, 'e': 53, 'N': 54, "'": 55, 'Q': 56, 'u': 57, ']': 58, 'P': 59, '?': 60, 'X': 61, 'a': 62, 'G': 63, ':': 64, '&': 65, 'I': 66, 'b': 67, 'R': 68, 'm': 69, '<PAD>': 0, '<S>': 1, '</S>': 2}


In [20]:
for seq_batch in data:
  for seq in seq_batch:
    print(seq)
    print("-----------------------------------")
    inp = tf.one_hot(seq, vocab_size, axis=-1)
    oh_seq = inp
    print(inp)
    to_chars = "".join([ind_to_ch[ind] for ind in seq.numpy()])
    print(to_chars)
    if input() == 'x':
      break
  break

tf.Tensor(
[ 1 25 68 66 35 12 64 44 25 22  5 33 48  7 16 39  2  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0 

In [0]:
n_h = 512

layers = [tf.keras.Input(shape=(seq_len, vocab_size), batch_size=128),
          tf.keras.layers.SimpleRNN(n_h, return_sequences=True, stateful=True),
          tf.keras.layers.Dense(vocab_size)]

model = tf.keras.Sequential(layers)


steps = 20*35000 // bs
opt = tf.optimizers.Adam()
loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

model.build()

all_vars = [vars for vars in model.trainable_variables]

In [0]:
@tf.function
def run_rnn_on_seq(seq_batch):
    model.reset_states()
    with tf.GradientTape() as tape:
        mask_len = [] 
        mask_len.append(tf.math.count_nonzero(seq, dtype=tf.float32)-1,)

        mask_len = tf.stack(mask_len)        
        mask = tf.sequence_mask(mask_len, seq_len, dtype=tf.float32)       
        

        oh_seq  = tf.one_hot(seq_batch, vocab_size, axis=-1)        
        y_actual = tf.roll(seq_batch, -1, 1)
        

        logits = model(oh_seq)        
        losses = loss_fn(y_actual, logits)
        
        
        losses = losses * mask        
        losses = tf.reduce_sum(losses, axis=1)    # sum up losses for each seq

        # average loss over batches
        xcent = tf.reduce_mean(tf.math.divide(losses,mask_len))   # dividing the sum of losses for each seq by the seq len

    grads = tape.gradient(xcent, all_vars)
    
    # this is gradient clipping
    glob_norm = tf.linalg.global_norm(grads)
    grads = [g/glob_norm for g in grads]
    
    opt.apply_gradients(zip(grads, all_vars))

    return xcent

In [19]:
for step, seqs in enumerate(data):
    xent_avg = run_rnn_on_seq(seqs)

    if not step % 100:
        print("Step: {} Loss: {}".format(step, xent_avg))

    if step > steps:
        break

Step: 0 Loss: 4.18389892578125
Step: 100 Loss: 0.5284740924835205
Step: 200 Loss: 0.4834766983985901
Step: 300 Loss: 0.48588138818740845
Step: 400 Loss: 0.49427294731140137
Step: 500 Loss: 0.46960213780403137
Step: 600 Loss: 0.4127671718597412
Step: 700 Loss: 0.4209989309310913
Step: 800 Loss: 0.40784698724746704
Step: 900 Loss: 0.41049912571907043
Step: 1000 Loss: 0.41542983055114746
Step: 1100 Loss: 0.4061402678489685
Step: 1200 Loss: 0.39543670415878296
Step: 1300 Loss: 0.3840625286102295
Step: 1400 Loss: 0.39147135615348816
Step: 1500 Loss: 0.3904823362827301
Step: 1600 Loss: 0.3794107437133789
Step: 1700 Loss: 0.4062483310699463
Step: 1800 Loss: 0.33503055572509766
Step: 1900 Loss: 0.3897464871406555
Step: 2000 Loss: 0.3149295449256897
Step: 2100 Loss: 0.43651893734931946
Step: 2200 Loss: 0.3580419719219208
Step: 2300 Loss: 0.3495141565799713
Step: 2400 Loss: 0.3237883746623993
Step: 2500 Loss: 0.2928924262523651
Step: 2600 Loss: 0.3386484384536743
Step: 2700 Loss: 0.3663747310638

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (128, 500, 512)           298496    
_________________________________________________________________
dense_1 (Dense)              (128, 500, 70)            35910     
Total params: 334,406
Trainable params: 334,406
Non-trainable params: 0
_________________________________________________________________


In [21]:
# create new model for generating text; batch size 1, input 1 char 

layers_gen = [tf.keras.Input(shape=(1,vocab_size), batch_size=1),
          tf.keras.layers.SimpleRNN(n_h, return_sequences=True, stateful=True),
          tf.keras.layers.Dense(vocab_size)]

model_gen = tf.keras.Sequential(layers_gen)

model_gen.build()

model_gen.set_weights(model.get_weights())

model_gen.summary() # same parameters, just different input/output shapes


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_4 (SimpleRNN)     (1, 1, 512)               298496    
_________________________________________________________________
dense_4 (Dense)              (1, 1, 70)                35910     
Total params: 334,406
Trainable params: 334,406
Non-trainable params: 0
_________________________________________________________________


In [22]:
def gen_seq(n_seq):
  for _ in range(n_seq):
    char = 1      
    txt = []
    while not char == 2:
      gen = tf.Variable([[tf.one_hot(char, vocab_size)]])
      out = model_gen(gen)
      probs = tf.reshape(tf.nn.softmax(out),[-1]).numpy()
      char = np.random.choice(vocab_size, p=probs)      
      txt.append(char)
      if char == 2:
        model_gen.reset_states()
        break

    print("".join([ind_to_ch[ind] for ind in txt]).replace("</S>", "\n\n"))

        
gen_seq(50)


DUKE:
How Farest thou seemshar, freely.


LUCIUS:
Here, let impurne you; for I mean to call;
And are the king, I know not where not that,
In prest as long have a block the string times.


LUCEMTENES:
And what thon worse, everent at the earth,
With streathes the law, belike into her strompe,
As year now of that four way?


HASTINGS:
I think imncal; and then here, robs eish no malice?


HORATIOC:
But I were blembers with uped by you,
Nay, that he is marrate, o' the comforty, shall say
Ofter thy showled princely juigna wish thy
lords; I prithee, successian, pureow unclary you,
Who lies friends, it were done meet. But I chier,
Pray our reports is mecarney drum like asit.


THESEUS:
Well, sir, we have made mescure much is dead!
Thy mastless and any, but too make at trait,
A deadly oving is ready
To sea, a's an exfects you indeed.


NERISSA:
He's viseared by toads: O, it ceftle
But oftrabelling noted, senve and dead?


Second Senator:
By foom one worthy onky:
In valfines ride her left of our

In [0]:
def get_seq_prob(seq):
  txt = tf.concat([[1], [ch_to_ind[ch] for ch in seq], [2]],axis=0)
  prob_lst = []

  char = 1
  for ch in txt.numpy():
    gen = tf.Variable([[tf.one_hot(ch, vocab_size)]])
    out = model_gen(gen)
    probs = tf.reshape(-tf.nn.log_softmax(out),[-1]).numpy()
    prob_lst.append(probs[ch])
  return tf.reduce_sum(prob_lst).numpy()


In [96]:
seq_1 = "CAESAR:\n\
Cowards die many times before their deaths;\n\
The valiant never taste of death but once.\n\
Of all the wonders that I yet have heard.\n\
It seems to me most strange that men should fear;\n\
Seeing that death, a necessary end,\n\
Will come when it will come.\n\
What say the augurers?"

get_seq_prob(seq_1)

2639.7817

In [97]:
# replacing vowels by x

seq_2 = "CXXSXR:\n\
Cxwxrds dxx mxny txmxs bxfxrx thxxr dxxths;\n\
Thx vxlxxnt nxvxr txstx xf dxxth bxt xncx.\n\
xf xll thx wxndxrs thxt x yxt hxvx hxxrd.\n\
xt sxxms tx mx mxst strxngx thxt mxn shxxld fxxr;\n\
Sxxxng thxt dxxth, x nxcxssxry xnd,\n\
Wxll cxmx whxn xt wxll cxmx.\n\
Whxt sxy thx xxgxrxrs?"

get_seq_prob(seq_2)

3186.086